# Import value of Magnetic rigidity from beam momentum

In [1]:
def MagneticRigidity (BeamMomentum):
    df = pd.read_csv("/eos/home-r/russogi/WINDOWS/Desktop/AD_ELENA_scripts_data/AD_MagneticModel/MagneticRigidity.csv")
    i = np.argmin( np.abs(df.p-BeamMomentum) )
    Magnetic_Rigidity = df.Brho.values[i]
    Momentum = df.p.values[i]
    return Magnetic_Rigidity, Momentum

# Define function to retrieve quadrupole strenghts

In [ ]:
def StrenghtsQuadrupoles (ChoiceOptics, TrimsStauts, BeamMomentum):
    ################################## TRIM STATUS ##########################
    if TrimsStauts == "OFF":
        state = 0
    else:
        state = 1
    ################################## BEAM MOMENTUM ##########################
    if BeamMomentum >= 2:
        energy = "HIGH"
    elif BeamMomentum <= 0.3:
        energy = "LOW"
    else:
        print("Beam momentum in the wrong unit, please enter in GeV/c")
    ################################## QUADRUPOLE STRENGHTS ##########################
    Deltak_Qmain1 = 0 * state
    Deltak_Qmain2 = 0 * state
    Deltak_QTrim1 = 0 * state # ! to adjust dispersion
    Deltak_QTrim2 = 0 * state # ! to adjust Tune V
    Deltak_QTrim3 = 0 * state
    Deltak_QTrim4 = 0 * state #! around e-cooler
    Deltak_QTrim5 = 0 * state
    
    if ChoiceOptics == "FromPavel":
        if energy == "HIGH":
            PyValkQFNS       =  0.558153759866                                          
            PyValkQDS        = -0.558153759866
            PyValkQFNECool   =  0.594999986705
            PyValkQDNEcool   = -0.36175160802
            PyValkQFN        = -0.558153759866 * -0.9206
            PyValkQDN        = -0.558153759866 *  0.8299
            PyValkQDC53      = -0.558153759866 *  0.8299
            PyValkQFC54      = -0.558153759866 * -0.9206
            PyValkQFW6       =  0.434013207981
            PyValkQFW8       =  0.560993621836
            # define strenghts of QFW7 as a function of the strenght in QFW9
            PyValkQDW9       = 0.7845*-0.526485397793
            PyValkQDW7       = -0.526485397793
        elif energy == "LOW":
            PyValkQFNS       =  0.58123                                          
            PyValkQDS        = -0.58123
            PyValkQFNECool   =  0.61605492304
            PyValkQDNEcool   = -0.362715221615
            PyValkQFN        = -0.58123 *  -0.9080
            PyValkQDN        = -0.58123 *  0.7944
            PyValkQDC53      = -0.58123 *  0.7944
            PyValkQFC54      = -0.58123 *  -0.9080
            PyValkQFW6       =  0.432112439772
            PyValkQFW8       =  0.557766251018
            # define strenghts of QFW7 as a function of the strenght in QFW9
            PyValkQDW9       = 0.7686*-0.53074784697
            PyValkQDW7       = -0.53074784697

        strenghts = [PyValkQFNS, PyValkQDS, PyValkQFNECool,PyValkQDNEcool,PyValkQFN,PyValkQDN,PyValkQDC53,
                     PyValkQFC54, PyValkQFW6, PyValkQFW8, PyValkQDW9, PyValkQDW7]

    elif ChoiceOptics == "FromCurrents":
        ### automatically loads the .csv file with the strengths for a given beam momentum
        df_currents = pd.read_csv(f"/eos/home-r/russogi/WINDOWS/Desktop/AD_ELENA_scripts_data/AD_MagneticModel/AD_StrenghtsQuadrupolesFor%sGeV.csv"%str(BeamMomentum))

        PyValkQFNS       =   df_currents["QFNS"].dropna().values[0] * (1 + Deltak_Qmain1) 
        
        PyValkQDS        =   df_currents["QDS"].dropna().values[0]  * (1 + Deltak_Qmain1)
        
        PyValkQFNECool   =   (df_currents["QFNS"].dropna().values[0] * (1 + Deltak_Qmain1) +
                              df_currents["QFNS"].dropna().values[-1]  * (1 + Deltak_QTrim4) )
        
        PyValkQDNEcool   = -(df_currents["QFNS"].dropna().values[1]) * (1 + Deltak_Qmain2) 
        
        PyValkQFN        =   df_currents["QFN"].dropna().values[0] * (1 + Deltak_Qmain1)
        
        PyValkQDN        =   df_currents["QDN"].dropna().values[0]  * (1 + Deltak_Qmain1)
        
        PyValkQDC53      =   df_currents["QDC"].dropna().values[0] * (1 + Deltak_Qmain1) 
        
        PyValkQFC54      =  ( df_currents["QFC"].dropna().values[0] * (1 + Deltak_Qmain1)  +
                              df_currents["QFC"].dropna().values[-1] * (1 + Deltak_QTrim5) )
        
        PyValkQFW6       =  ( df_currents["QFW6"].dropna().values[0] * (1 + Deltak_Qmain1) +
                              df_currents["QFW6"].dropna().values[-1] * (1 + Deltak_QTrim1))             
        
        PyValkQFW8       =  ( df_currents["QFW8"].dropna().values[0] * (1 + Deltak_Qmain1) + 
                             df_currents["QFW8"].dropna().values[-1]   * (1 + Deltak_QTrim3) )

        # define strenghts of QFW7 as a function of the strenght in QFW9
        PyValkQDW9       = ( df_currents["QDW9"].dropna().values[0] * (1 + Deltak_Qmain1)  + 
                            df_currents["QDW9"].dropna().values[-1]  * (1 + Deltak_QTrim2) )
        
        PyValkQDW7       = ( df_currents["QDW7"].dropna().values[0] * (1 + Deltak_Qmain1)  +  
                            df_currents["QDW7"].dropna().values[-1]  * (1 + Deltak_QTrim2) )

        strenghts = [PyValkQFNS, PyValkQDS, PyValkQFNECool,PyValkQDNEcool,PyValkQFN,PyValkQDN,PyValkQDC53,
                     PyValkQFC54, PyValkQFW6, PyValkQFW8, PyValkQDW9, PyValkQDW7]
    return strenghts

# Dipoles trims

In [ ]:
def DipoleTrims (BeamMomentum, AdjustTrims):
    if AdjustTrims == False:
        state = 0
    else:
        state = 1

    DeltaTrim_5_6   = 0 * state
    DeltaTrim_8_9   = 0 * state
    DeltaTrim_11_12 = 0 * state
    DeltaTrim_17_18 = 0 * state
    DeltaTrim_20_21 = 0 * state
    DeltaTrim_23_24 = 0 * state
    DeltaTrim_33_34 = 0 * state
    DeltaTrim_36_37 = 0 * state
    DeltaTrim_39_40 = 0 * state
    DeltaTrim_45_46 = 0 * state
    DeltaTrim_48_49 = 0 * state
    DeltaTrim_51_52 = 0 * state
        
    df = pd.read_csv("/eos/home-r/russogi/WINDOWS/Desktop/AD_ELENA_scripts_data/AD_MagneticModel/MagneticRigidity.csv")
    i = np.argmin( np.abs(df.p-BeamMomentum) )
    
    PyKBHZTRIM   =    [0.000464380099999999 , 0.00040472109999999996, 0, 0][i] 
    PyKBHZTR0506 =    [-0.0002432109 , 0.0012811341, 0, 0][i]                   * (1 + DeltaTrim_5_6)
    PyKBHZTR0809 =    [0.0007231061, 0.000187596, 0, 0][i]                      * (1 + DeltaTrim_8_9)
    PyKBHZTR1112 =    [0.0001839331, 0.0004126341, 0, 0][i]                     * (1 + DeltaTrim_11_12)
    PyKBHZTR1718 =    [-0.0004132339, -0.0007106259, 0, 0][i]                   * (1 + DeltaTrim_17_18)
    PyKBHZTR2021 =    [0.0010593001, 0.0014532901000000001, 0, 0][i]            * (1 + DeltaTrim_20_21)
    PyKBHZTR2324 =    [0.0000630230999999999, 5.1145100000000006e-05, 0, 0][i]  * (1 + DeltaTrim_23_24)
    PyKBHZTR3334 =    [0.0007204311, 0.0015200681, 0, 0][i]                     * (1 + DeltaTrim_33_34)
    PyKBHZTR3637 =    [0.0007782111, -0.0006363209, 0, 0][i]                    * (1 + DeltaTrim_36_37)
    PyKBHZTR3940 =    [-0.0005899979, 0.0007131351, 0, 0][i]                    * (1 + DeltaTrim_39_40)
    PyKBHZTR4546 =    [0.0007003151, 0.0009567011, 0, 0][i]                     * (1 + DeltaTrim_45_46)
    PyKBHZTR4849 =    [0.0010593001, 0.0013969341, 0, 0][i]                     * (1 + DeltaTrim_48_49)
    PyKBHZTR5152 =    [-0.0009258709, -0.0007430499, 0, 0][i]                   * (1 + DeltaTrim_51_52)
    
    trims_value = [ PyKBHZTRIM, PyKBHZTR0506, PyKBHZTR0809, PyKBHZTR1112, PyKBHZTR1718, PyKBHZTR2021, PyKBHZTR2324,
                    PyKBHZTR3334, PyKBHZTR3637, PyKBHZTR3940, PyKBHZTR4546, PyKBHZTR4849, PyKBHZTR5152]
    return trims_value

# set up MAD-X environment

In [ ]:
# # Setting up the MAD-X environment
def mad_setup(BeamMomentum, BRHO_VALUE, BeamEnergy, Beam_H_size, Beam_V_size, EnergySpread, NumberParticles, ValkQFNS, ValkQDS, 
              ValkQFNECool, ValkQDNEcool, ValkQFN, ValkQDN, ValkQDC53, ValkQFC54, ValkQFW6, ValkQFW8, ValkQDW7, ValkQDW9,
             KBHZTRIM,KBHZTR0506,KBHZTR0809,KBHZTR1112,KBHZTR1718,KBHZTR2021,KBHZTR2324,KBHZTR3334,KBHZTR3637,
             KBHZTR3940,KBHZTR4546,KBHZTR4849, KBHZTR5152):
    print(BeamMomentum)
    
    madx = Madx(stdout=True, command_log = "log.madx" ) 
    
    s = '''
    !CALL, FILE = "/eos/home-r/russogi/WINDOWS/Desktop/AD_ELENA_scripts_data/Scripts/acc-models-ad/scenarios/studyEC/studyEC_continue.str";!file for strenghts (quads, ...)
    CALL, FILE = "/eos/home-r/russogi/WINDOWS/Desktop/AD_ELENA_scripts_data/Scripts/acc-models-ad/ad.seq";!file for sequence of elements
    CALL, FILE = "/eos/home-r/russogi/WINDOWS/Desktop/AD_ELENA_scripts_data/Scripts/acc-models-ad/ad.dbx";!file for apertures
    CALL, FILE = "/eos/home-r/russogi/WINDOWS/Desktop/AD_ELENA_scripts_data/Scripts/acc-models-ad/tools/minimum_split_and_add_BPMs.madx";
    CALL, FILE = "/eos/home-r/russogi/WINDOWS/Desktop/AD_ELENA_scripts_data/Scripts/acc-models-ad/tools/additional_split.madx";
    CALL, FILE = "/eos/home-r/russogi/WINDOWS/Desktop/AD_ELENA_scripts_data/Scripts/acc-models-ad/tools/split_dipoles.madx";
    '''
    madx.input(s)
    
    if BeamEnergy >3:
        s = '''
        CALL, FILE = "/eos/home-r/russogi/WINDOWS/Desktop/AD_ELENA_scripts_data/Scripts/acc-models-ad/scenarios/highenergy/highenergy.str";!file for strenghts (quads, ...)
        '''
        madx.input(s)
    else:
        s = '''
        CALL, FILE = "/eos/home-r/russogi/WINDOWS/Desktop/AD_ELENA_scripts_data/Scripts/acc-models-ad/scenarios/lowenergy/lowenergy.str";
        '''
        madx.input(s)
    
    s = f'''
    BEAM, PARTICLE=ANTIPROTON, PC = {BeamEnergy}, ex={Beam_H_size},ey={Beam_V_size}, sige={EnergySpread}, NPART={NumberParticles};
    BRHO = {BRHO_VALUE};
    MOMENTUM = {BeamMomentum};
    '''
    madx.input(s)
    
    s = '''
    use, sequence = AD;
    '''
    madx.input(s)
    
    s = f'''
        ENABLE_QKICK := 0;
        ENABLE_DKICK := 0;
        ENABLE_Q_SEXT := 0;
    '''
    madx.input(s)
    
    s = f'''
    
    K.BHZTRIM = {KBHZTRIM};
    K.BHZTR05.06 := {KBHZTR0506};
    K.BHZTR08.09 := {KBHZTR0809};
    K.BHZTR11.12 := {KBHZTR1112};
    K.BHZTR17.18 := {KBHZTR1718};
    K.BHZTR20.21 := {KBHZTR2021};
    K.BHZTR23.24 := {KBHZTR2324};
    K.BHZTR33.34 := {KBHZTR3334};
    K.BHZTR36.37 := {KBHZTR3637};
    K.BHZTR39.40 := {KBHZTR3940};
    K.BHZTR45.46 := {KBHZTR4546};
    K.BHZTR48.49 := {KBHZTR4849};
    K.BHZTR51.52 := {KBHZTR5152};
    '''
    madx.input(s)
    
    s = f'''
    kQFNS       := {ValkQFNS};
    kQDS        := {ValkQDS};
    kQFNECool   := {ValkQFNECool};
    kQDNEcool   := {ValkQDNEcool};
    kQFN        := {ValkQFN};
    kQDN        := {ValkQDN};
    kQDC53      := {ValkQDC53};
    kQFC54      := {ValkQFC54};
    kQFW6       := {ValkQFW6};
    kQFW8       := {ValkQFW8};
    kQDW7       := {ValkQDW7};
    kQDW9       := {ValkQDW9};
    '''
    madx.input(s)
    
    #s = '''
    #DR.DVT1304: VKICKER, KICK := 0; !old -> 0.0001, new -> -0.0010526
    #DR.DVT0208: VKICKER, KICK := 0; !old ->0.0003, new -> 0.001281
    #DR.DVT5408: VKICKER, KICK := 0; !old -> 0.000404, new -> 0.0003854
    #'''
    #madx.input(s)
    
    s = '''
    OBSERVATION_POINT: MARKER;
    
    seqedit,sequence = AD;
            flatten;
            INSTALL, ELEMENT=Obs_point, CLASS=OBSERVATION_POINT, AT=40.85420;
            INSTALL, ELEMENT=Obs_point_StartCooler, CLASS=OBSERVATION_POINT, AT=85.776800;
            INSTALL, ELEMENT=Obs_point_EndCooler, CLASS=OBSERVATION_POINT, AT=96.656000;
            INSTALL, ELEMENT=Obs_point_pickups, CLASS=OBSERVATION_POINT, AT=101.513900;
            INSTALL, ELEMENT=Obs_point_kickers, CLASS=OBSERVATION_POINT, AT=10.297500;
           flatten;
    endedit;

    use, sequence = AD;
    '''
    madx.input(s)
    
    return madx